<a href="https://colab.research.google.com/github/jharviy/fcc_machinelearning/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
# try:
#   # %tensorflow_version only exists in Colab.
#   !pip install tf-nightly
# except Exception:
#   pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# !pip install tensorflow-datasets
# import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.12.0


In [2]:
from tensorflow.keras.preprocessing.text import one_hot   #Deprecated

In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-07-04 08:19:28--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.007s  

2023-07-04 08:19:28 (48.0 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2023-07-04 08:19:28--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [4]:
# Read tsv file using read_table, this defaults to separator of ‘\t’ (tab) then assign column names.
columns = ["label", "sms"]
train_df = pd.read_table(train_file_path, names = columns)
test_df = pd.read_table(test_file_path, names = columns)


In [5]:
# SAMPLE TRAIN DATA
train_df.head(5)

,label,sms
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [6]:
# SAMPLE TEST DATA
test_df.sample(5)

,label,sms
1077,ham,"haha i heard that, text me when you're around"
881,ham,i'll be late...
307,ham,no. i.ll meet you in the library
1249,ham,"go until jurong point, crazy.. available only ..."
1206,ham,"hey elaine, is today's meeting still on?"


In [7]:
# DATA PREPROCESSING

# Encode labels with "0" for "ham" (non-spam) and "1" for "spam" message. Then convert to np.array for input in model
train_labels = np.array(train_df.pop("label").map({"ham":0,"spam":1}).tolist())
test_labels = np.array(test_df.pop("label").map({"ham":0,"spam":1}).tolist())


# For counting of unique words. NOTE: Not all are actual words, this depends on sms data and some are mispelled.
vocabulary = set()
for message in train_df["sms"].tolist():
  for word in message.split():
    vocabulary.add(word)  # Stores only unique values on a set

VOCAB_SIZE = len(vocabulary)  # Count of unique words in the training data
#MAX_LENGTH = len(max(train_sms, key=lambda x: len(x.split())).split())          # Max length will be equal to the longest message in the training data (171)
MAX_LENGTH = 250


# HASH each word in each sample into numeric (integer). One number for each unique word.
encoded_train_message = [one_hot(d, VOCAB_SIZE) for d in train_df["sms"].tolist()]
encoded_test_message = [one_hot(d, VOCAB_SIZE) for d in test_df["sms"].tolist()]


# PAD 0's to the encoded messages to reach the MAX_LENGTH. This ensures equal length of input data for each sample
padded_train_message = keras.utils.pad_sequences(encoded_train_message, maxlen=MAX_LENGTH, padding='post')
padded_test_message = keras.utils.pad_sequences(encoded_test_message, maxlen=MAX_LENGTH, padding='post')

In [8]:
# Make a model
def make_model():
  model = keras.Sequential()
  model.add(keras.layers.Embedding(VOCAB_SIZE, 32, input_length = MAX_LENGTH))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
  return model


model = make_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 32)           362560    
                                                                 
 flatten (Flatten)           (None, 8000)              0         
                                                                 
 dense (Dense)               (None, 1)                 8001      
                                                                 
Total params: 370,561
Trainable params: 370,561
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Train data for 20 epochs
model.fit(padded_train_message,
          train_labels,
          validation_data = (padded_test_message, test_labels),
          epochs = 20,
          )

Epoch 1/20
131/131 [==============================] - 3s 15ms/step - loss: 0.3123 - acc: 0.8830 - val_loss: 0.1520 - val_acc: 0.9555
Epoch 2/20
131/131 [==============================] - 1s 9ms/step - loss: 0.0861 - acc: 0.9768 - val_loss: 0.0727 - val_acc: 0.9806
Epoch 3/20
131/131 [==============================] - 1s 9ms/step - loss: 0.0437 - acc: 0.9888 - val_loss: 0.0563 - val_acc: 0.9835
Epoch 4/20
131/131 [==============================] - 1s 9ms/step - loss: 0.0276 - acc: 0.9921 - val_loss: 0.0471 - val_acc: 0.9864
Epoch 5/20
131/131 [==============================] - 1s 9ms/step - loss: 0.0178 - acc: 0.9952 - val_loss: 0.0433 - val_acc: 0.9885
Epoch 6/20
131/131 [==============================] - 1s 11ms/step - loss: 0.0121 - acc: 0.9971 - val_loss: 0.0413 - val_acc: 0.9878
Epoch 7/20
131/131 [==============================] - 1s 8ms/step - loss: 0.0084 - acc: 0.9988 - val_loss: 0.0406 - val_acc: 0.9864
Epoch 8/20
131/131 [==============================] - 1s 6ms/step - loss: 

In [10]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  # ENCODE and PAD 0's to the message to the same length used in the model
  encoded_message = [one_hot(pred_text, VOCAB_SIZE)]
  padded_message = keras.utils.pad_sequences(encoded_message, maxlen=MAX_LENGTH, padding='post')

  # Use predict to get probability of message being a spam
  probability = model.predict(padded_message)[0][0]
  f = lambda x: "ham" if x == 0 else "spam"

  prediction = [probability, f(np.round(probability))]
  #print(prediction)
  return (prediction)


pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 65ms/step
[0.00023364523, 'ham']


In [11]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 16ms/step
You passed the challenge. Great job!
